<a href="https://colab.research.google.com/github/Ximena5745/Desempeno_Institucional/blob/main/Consolidado_Ind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import glob
import re
import ast
import pandas as pd
from html import unescape

In [9]:
!git clone https://github.com/Ximena5745/Desempeno_Institucional.git

Cloning into 'Desempeno_Institucional'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 1.19 MiB | 18.98 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [10]:
os.chdir("/content/Desempeno_Institucional/Data")

In [11]:
# 3) Cargar todos los Excel del directorio, excluyendo cualquier archivo de consolidado previo
archivos_excel = [f for f in (glob.glob("*.xlsx") + glob.glob("*.xls")) if "consolidado" not in f.lower()]


In [15]:
dfs = []
for archivo in archivos_excel:
    try:
        df_tmp = pd.read_excel(archivo)
        df_tmp["Archivo"] = archivo
        dfs.append(df_tmp)
    except Exception as e:
        print(f"⚠️ No se pudo leer {archivo}: {e}")

if not dfs:
    raise RuntimeError("No se encontraron archivos Excel para consolidar.")

# 4) Unir y estandarizar columnas (evitar autoreplicado y duplicados)
df_todos = pd.concat(dfs, ignore_index=True).drop_duplicates()
# Normalizar nombres de columnas a minúsculas para operar sin problemas de mayúsculas
df_todos.columns = df_todos.columns.str.strip().str.lower()

# 5) Limpieza pedida
# 5.1 Eliminar columnas si existen
cols_a_eliminar = ["dependencia", "incidencia", "objectivos_relacionados"]
df_todos = df_todos.drop(columns=[c for c in cols_a_eliminar if c in df_todos.columns], errors="ignore")

# 5.2 Eliminar filas con 'fecha' vacía/nula
if "fecha" in df_todos.columns:
    df_todos = df_todos[df_todos["fecha"].notna() & (df_todos["fecha"].astype(str).str.strip() != "")]

# 5.3 Corregir clasificacion "Estrat&eacute;gico" → "Estratégico" (y decodificar entidades HTML por si hay más)
if "clasificacion" in df_todos.columns:
    df_todos["clasificacion"] = df_todos["clasificacion"].astype(str).map(lambda s: unescape(s))
    df_todos["clasificacion"] = df_todos["clasificacion"].replace({"Estrat&eacute;gico": "Estratégico"})

# -------------------------
# 6) EXPANSIÓN DE 'variables' CUANDO tipo == 'Serie Unica'
#    Soporta dos formatos:
#    a) Lista de diccionarios como string: "[{'valor': 97.5, 'nombre': 'X', 'simbolo': 'Y'}, ...]"
#    b) Cadena con bloques {} consecutivos (fallback)
# -------------------------

def parse_variables_cell(x):
    """Devuelve una lista de dicts con llaves: valor, nombre, simbolo."""
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        # Ya es lista de dicts
        out = []
        for item in x:
            if isinstance(item, dict):
                out.append({
                    "valor": item.get("valor"),
                    "nombre": item.get("nombre"),
                    "simbolo": item.get("simbolo")
                })
        return out

    s = str(x).strip()
    if s == "" or s.lower() in {"nan", "none", "null", "[]"}:
        return []

    # Intento 1: interpretar como literal de Python
    try:
        pyobj = ast.literal_eval(
            s.replace("null", "None").replace("true", "True").replace("false", "False")
        )
        if isinstance(pyobj, list):
            out = []
            for item in pyobj:
                if isinstance(item, dict):
                    out.append({
                        "valor": item.get("valor"),
                        "nombre": item.get("nombre"),
                        "simbolo": item.get("simbolo")
                    })
            return out
    except Exception:
        pass

    # Intento 2 (fallback): extraer bloques {...} y localizar campos
    bloques = re.findall(r"\{[^{}]*\}", s)
    out = []
    for b in bloques:
        # valor (acepta coma decimal)
        m_val = re.search(r"[\'\"]?valor[\'\"]?\s*:\s*([\-+]?\d+(?:[.,]\d+)?)", b, flags=re.IGNORECASE)
        m_nom = re.search(r"[\'\"]?nombre[\'\"]?\s*:\s*[\'\"](.*?)[\'\"]", b, flags=re.IGNORECASE)
        m_sim = re.search(r"[\'\"]?simbolo[\'\"]?\s*:\s*[\'\"](.*?)[\'\"]", b, flags=re.IGNORECASE)

        valor = None
        if m_val:
            v = m_val.group(1).replace(",", ".")
            try:
                valor = float(v)
            except Exception:
                valor = v  # déjalo como texto si no se puede convertir

        nombre = m_nom.group(1) if m_nom else None
        simbolo = m_sim.group(1) if m_sim else None

        if any([m_val, m_nom, m_sim]):
            out.append({"valor": valor, "nombre": nombre, "simbolo": simbolo})

    return out

# Aplicar solo a 'serie unica'
if "tipo" in df_todos.columns and "variables" in df_todos.columns:
    mask_serie = df_todos["tipo"].astype(str).str.strip().str.lower() == "serie unica"
    if mask_serie.any():
        # Parsear lista de variables por fila
        vars_list = df_todos.loc[mask_serie, "variables"].apply(parse_variables_cell)

        # Número máximo de variables en cualquier fila Serie Unica
        max_vars = int(vars_list.map(lambda lst: len(lst) if isinstance(lst, list) else 0).max())

        # Crear columnas dinámicas
        new_cols = []
        for i in range(1, max_vars + 1):
            for base in ("Variable_Valor", "Variable_Nombre", "Variable_Simbolo"):
                col = f"{base}_{i}"
                new_cols.append(col)
                if col not in df_todos.columns:
                    df_todos[col] = pd.NA

        # Construir DataFrame expandido y asignar por índice
        def expand_row(lst, max_n):
            data = {}
            if not isinstance(lst, list):
                return pd.Series({c: pd.NA for c in new_cols})
            for i in range(min(max_n, len(lst))):
                item = lst[i] if isinstance(lst[i], dict) else {}
                data[f"Variable_Valor_{i+1}"]  = item.get("valor", pd.NA)
                data[f"Variable_Nombre_{i+1}"] = item.get("nombre", pd.NA)
                data[f"Variable_Simbolo_{i+1}"]= item.get("simbolo", pd.NA)
            # Completar faltantes si la fila tiene menos variables que el máximo
            for i in range(len(lst)+1, max_n+1):
                data.setdefault(f"Variable_Valor_{i}", pd.NA)
                data.setdefault(f"Variable_Nombre_{i}", pd.NA)
                data.setdefault(f"Variable_Simbolo_{i}", pd.NA)
            return pd.Series(data)

        expanded = vars_list.apply(lambda lst: expand_row(lst, max_vars))
        df_todos.loc[mask_serie, expanded.columns] = expanded.values

# 7) Guardar resultado
df_todos.to_excel("Consolidado.xlsx", index=False)
print("✅ Consolidado.xlsx generado con columnas de variables para 'Serie Unica' (si aplica).")

✅ Consolidado.xlsx generado con columnas de variables para 'Serie Unica' (si aplica).
